In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format,year,col,to_timestamp


In [0]:
amazon_df = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv",header = True, inferSchema = True)
amazon_df.show()
amazon_df.printSchema()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|
|7792c28f-2f9b-48b...|Adult Womens Sexy...|       adult|   shoes|  a

In [0]:
amazon_df.createOrReplaceTempView("TDA_Table")

In [0]:
amazon_df1 = amazon_df.withColumn("InvoiceDate",to_timestamp(col("InvoiceDate"))).withColumn("Month",date_format(col("InvoiceDate"),"M"))
amazon_df1.select("Brand","SubCategory","Month").show()

+------------+-------------------+-----+
|       Brand|        SubCategory|Month|
+------------+-------------------+-----+
|      landau|  all women's shoes|    1|
|       adult|  all women's shoes|    1|
|    woolrich|           athletic|    1|
|       style|      women's shoes|    1|
|         two|  all women's shoes|    1|
|     michael|          necklaces|    1|
|        bali|    shoes & jewelry|    1|
|dr. scholl's|  all women's shoes|    1|
| coloriffics|women's accessories|    1|
|  timberland|       shoes & bags|    1|
|      bamboo|  all women's shoes|    1|
|   evolatree|women's accessories|    1|
|         bar|women's accessories|    1|
|     michael|  all women's shoes|    1|
|      excell|   women's clothing|    1|
|        vans|      women's jeans|    1|
|         nwt|      women's shoes|    1|
|        toms|  all women's shoes|    1|
|       earth|              boots|    1|
|     tretorn|  all women's shoes|    1|
+------------+-------------------+-----+
only showing top

In [0]:
amazon_df1.select("Month").distinct().show()

+-----+
|Month|
+-----+
|    3|
|    5|
|    6|
|    1|
|    4|
|    2|
|    7|
|   11|
|    8|
|    9|
|   10|
|   12|
+-----+



In [0]:
#a.----------	Find the top five performing mobile brands for the months of November, December and January r each year.
amazon_df1 = amazon_df.withColumn("Year",year(amazon_df.InvoiceDate)).withColumn("Month",month(amazon_df.InvoiceDate)).withColumn('quarter',quarter(amazon_df.InvoiceDate))
amazon_df1.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|Year|Month|quarter|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|2015|    1| 

In [0]:
amazon_df1.createOrReplaceTempView("TDA_Table")

In [0]:
spark.sql("select Brand, Month, Year, sum(Quantity) as Total_sold from TDA_Table where SubCategory='mobile' and Month in (11,12,1) group by Year, Month, Brand order by Total_sold DESC limit 5").show()

+-------+-----+----+----------+
|  Brand|Month|Year|Total_sold|
+-------+-----+----+----------+
|samsung|   12|2015|       517|
|samsung|   11|2015|       448|
|samsung|    1|2018|       434|
|samsung|   11|2016|       410|
|samsung|    1|2017|       395|
+-------+-----+----+----------+



In [0]:
amazon_df1.select("Month").distinct().show()


+-----+
|Month|
+-----+
|    1|
|    6|
|    3|
|    5|
|    4|
|    2|
|    9|
|    8|
|    7|
|   10|
|   11|
|   12|
+-----+



In [0]:
#b.	Find the top five selling products for the months of November, December and January for each year.
spark.sql("select Brand,SubCategory,sum(Quantity) as Total_sold from TDA_Table where Month in (11,12,1) group by Brand, SubCategory order by Total_sold DESC limit 5").show()

+-------+-----------+----------+
|  Brand|SubCategory|Total_sold|
+-------+-----------+----------+
|samsung|     mobile|      3974|
|   nike|men's shoes|      2695|
|    blu|     mobile|      1892|
|  apple|     mobile|      1818|
|  nokia|     mobile|      1467|
+-------+-----------+----------+



In [0]:
# 2.a. Number of products sold along with revenue aggregated quarterly
spark.sql("select Year, quarter,sum(Quantity) as Num_Product_Sold,sum(TransactionAmount) as Total_Revenue from TDA_Table group by Year, quarter order by Year,quarter").show()

+----+-------+----------------+-------------------+
|Year|quarter|Num_Product_Sold|      Total_Revenue|
+----+-------+----------------+-------------------+
|2015|      1|           42693|       3.08548555E8|
|2015|      2|           42069|2.23020302445285E14|
|2015|      3|           43224|       3.17578576E8|
|2015|      4|           45636|       3.22182882E8|
|2016|      1|           49125|2.23020349928595E14|
|2016|      2|           41872|4.46040332280323E14|
|2016|      3|           44353|6.69060331721781E14|
|2016|      4|           42684|4.46040321418626E14|
|2017|      1|           44917|       3.23175489E8|
|2017|      2|           44283|       3.45020711E8|
|2017|      3|           40429|       2.91143349E8|
|2017|      4|           45000|4.46040344731496E14|
|2018|      1|           42619|6.69060301392097E14|
|2018|      2|            3905|        3.0780533E7|
+----+-------+----------------+-------------------+



In [0]:
#b. Number of products sold in each category aggregated quarterly
spark.sql("select SubCategory,year,quarter,sum(Quantity) as Num_Product_Sold from TDA_Table group by year,SubCategory,quarter order by Num_Product_Sold desc").show()

+-----------------+----+-------+----------------+
|      SubCategory|year|quarter|Num_Product_Sold|
+-----------------+----+-------+----------------+
|           mobile|2015|      4|            8374|
|all women's shoes|2016|      1|            8248|
|all women's shoes|2017|      4|            7947|
|all women's shoes|2016|      3|            7689|
|           mobile|2017|      2|            7423|
|all women's shoes|2017|      1|            7368|
|all women's shoes|2015|      3|            7278|
|all women's shoes|2015|      4|            7190|
|all women's shoes|2015|      1|            7155|
|all women's shoes|2016|      2|            7125|
|      men's shoes|2016|      1|            6921|
|all women's shoes|2015|      2|            6908|
|           mobile|2016|      3|            6869|
|all women's shoes|2017|      2|            6833|
|           mobile|2016|      4|            6810|
|           mobile|2017|      1|            6767|
|           mobile|2016|      1|            6733|


In [0]:
#c. Number of products sold for each brand aggregated quarterly.
spark.sql("select Brand,year,quarter,sum(Quantity) as Num_Product_Sold from TDA_Table group by year,Brand,quarter order by Num_Product_Sold desc").show()

+-------+----+-------+----------------+
|  Brand|year|quarter|Num_Product_Sold|
+-------+----+-------+----------------+
|   nike|2016|      1|            2902|
|   nike|2017|      1|            2743|
|   nike|2017|      2|            2731|
|   nike|2016|      4|            2729|
|   nike|2017|      4|            2645|
|   nike|2015|      1|            2642|
|   nike|2018|      1|            2623|
|   nike|2016|      3|            2544|
|   nike|2015|      3|            2520|
|   nike|2015|      4|            2520|
|   nike|2016|      2|            2399|
|   nike|2015|      2|            2336|
|   nike|2017|      3|            2330|
|samsung|2015|      4|            1536|
|samsung|2017|      2|            1463|
|samsung|2016|      3|            1434|
|samsung|2017|      3|            1349|
|samsung|2016|      4|            1334|
|samsung|2017|      1|            1302|
|samsung|2016|      1|            1237|
+-------+----+-------+----------------+
only showing top 20 rows



In [0]:
###  3.#a. Top 5 profit margin generating products for a given month.
spark.sql("select Brand,SubCategory,(Marginpercentage-Discountpercentage) as profit,Month,year from TDA_Table group by year,SubCategory,Brand,Month,(Marginpercentage-Discountpercentage) order by profit desc").show()

+-------------+--------------------+------+-----+----+
|        Brand|         SubCategory|profit|Month|year|
+-------------+--------------------+------+-----+----+
|      express|   all women's shoes|    20|    2|2015|
|         geox|     all men's shoes|    20|    3|2015|
|        sorel|     shoes & jewelry|    20|    2|2015|
|          rdx|         men's shoes|    20|    1|2015|
|         nike|   all women's shoes|    20|    3|2015|
|         reef|         men's shoes|    20|    4|2015|
|    dreamgirl|   all women's shoes|    20|    3|2015|
|       vision|               jeans|    20|    5|2015|
|          blu|              mobile|    20|    3|2015|
|       azzuro|         men's boots|    20|    1|2015|
|bcbgeneration|other wholesale w...|    20|    3|2015|
|       deluxe|       women's shoes|    20|    5|2015|
|     original|         men's shoes|    20|    6|2015|
|     academie|         hats & caps|    20|    1|2015|
|        crocs|    petite swimsuits|    20|    6|2015|
|       ja

In [0]:
#4.a. Brand wise segregation of best performing categories profits wise
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from TDA_Table group by Brand,Category,(Marginpercentage-Discountpercentage) order by profit desc").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|               peter|   clothing|    20|
|             kenwood|electronics|    20|
|            castelli|   clothing|    20|
|                 dvs|   clothing|    20|
|               annie|   clothing|    20|
|              saddle|   clothing|    20|
|          underwraps|   clothing|    20|
|           haflinger|   clothing|    20|
|                 jbu|   clothing|    20|
|            muckboot|   clothing|    20|
|            siriusxm|electronics|    20|
|            funtasma|   clothing|    20|
|               perry|   clothing|    20|
|               donna|      shoes|    20|
|              noriko|   clothing|    20|
|                tumi|electronics|    20|
|                 cet|   clothing|    20|
|               nicki|      shoes|    20|
|yu&yu soccer men'...|   clothing|    20|
|               saint|   clothing|    20|
+--------------------+-----------+

In [0]:
#4.b. Brand wise segregation of least performing categories orders wise
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from TDA_Table group by Brand,Category,(Marginpercentage-Discountpercentage) order by profit ").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|           alexander|   clothing|   -50|
|               ridge|   clothing|   -50|
|               bgood|   clothing|   -50|
|             pantech|electronics|   -50|
|            alliance|   clothing|   -50|
|             bellini|   clothing|   -50|
|             natural|   clothing|   -50|
|         ellie shoes|   clothing|   -50|
|                five|   clothing|   -50|
|fruit of the loom...|   clothing|   -50|
|                easy|   clothing|   -50|
|              report|   clothing|   -50|
|                lowa|   clothing|   -50|
|                 nwt|   clothing|   -50|
|             emporio|   clothing|   -50|
|            siriusxm|electronics|   -50|
|            onitsuka|   clothing|   -50|
|             dickies|   clothing|   -50|
|               bally|   clothing|   -50|
|                moda|   clothing|   -50|
+--------------------+-----------+

In [0]:
#5.a. Find the top five selling products in a given month
spark.sql("select SubCategory,sum(Quantity) as Total_sold from TDA_Table group by SubCategory order by Total_sold desc").show(5)

+-----------------+----------+
|      SubCategory|Total_sold|
+-----------------+----------+
|all women's shoes|     92917|
|           mobile|     87843|
|      men's shoes|     79308|
|  shoes & jewelry|     33735|
|    women's shoes|     27424|
+-----------------+----------+
only showing top 5 rows



In [0]:
#5.b.	Find the top five products having the highest margin in a given month
spark.sql("select SubCategory,Marginpercentage,Month,Year from TDA_Table group by SubCategory,Marginpercentage,Month,Year order by Marginpercentage desc").show()

+--------------------+----------------+-----+----+
|         SubCategory|Marginpercentage|Month|Year|
+--------------------+----------------+-----+----+
|see more under ar...|              30|    4|2015|
|             walking|              30|    2|2015|
|     shoes & jewelry|              30|    5|2015|
|      women's skirts|              30|    4|2015|
|           moccasins|              30|    5|2015|
|      men's clothing|              30|    4|2015|
|see more samsung ...|              30|    5|2015|
|women's coats & j...|              30|    1|2015|
|               women|              30|    6|2015|
|            slippers|              30|    4|2015|
|             apparel|              30|    6|2015|
|  women's sunglasses|              30|    3|2015|
|              skirts|              30|    2|2015|
|       lawn & garden|              30|    2|2015|
|               youth|              30|    3|2015|
|                bras|              30|    5|2015|
|women's anniversa...|         

In [0]:
#5.#c. Find the five lowest selling products in a given month.
spark.sql("select SubCategory,Marginpercentage,Month,Year from TDA_Table group by SubCategory,Marginpercentage,Month,Year order by Marginpercentage ").show()

+--------------------+----------------+-----+----+
|         SubCategory|Marginpercentage|Month|Year|
+--------------------+----------------+-----+----+
|   bags & briefcases|             -20|    3|2015|
|  martial arts & mma|             -20|    3|2015|
|         accessories|             -20|    3|2015|
|see more vans aut...|             -20|    2|2015|
|  women's sunglasses|             -20|    2|2015|
|               jeans|             -20|    2|2015|
|               women|             -20|    3|2015|
|       women's boots|             -20|    3|2015|
|see more rock rev...|             -20|    3|2015|
|men's socks & und...|             -20|    6|2015|
|               boots|             -20|    4|2015|
|    shoes & handbags|             -20|    1|2015|
|       women's shoes|             -20|    5|2015|
|see more just kee...|             -20|    6|2015|
|            swimwear|             -20|    6|2015|
|               boots|             -20|    6|2015|
|               shoes|         